Rading text from files

In [0]:
from google.colab import files

uploaded = files.upload()

In [0]:
query = open('d_query.txt').read()
d1 = open('d1.txt').read()
d2 = open('d2.txt').read()
d3 = open('d3.txt').read()
d4 = open('d4.txt').read()
d5 = open('d5.txt').read()


vocab = d1 + d2+ d3+ d4+ d5
print(vocab)

Tokenize

In [0]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [0]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [0]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    data = np.char.replace(data, '\"', '')
    return data

In [0]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [0]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data) 
    data = remove_stop_words(data)
    data = remove_punctuation(data)
    return data

In [0]:
def convert_lower_case(data):
    return np.char.lower(data)


In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [0]:
processed_text = [] 
processed_text.append(word_tokenize(str(preprocess(d1))))
processed_text.append(word_tokenize(str(preprocess(d2))))
processed_text.append(word_tokenize(str(preprocess(d3))))
processed_text.append(word_tokenize(str(preprocess(d4))))
processed_text.append(word_tokenize(str(preprocess(d5))))
processed_text.append(word_tokenize(str(preprocess(query))))

d_query_tokens = [] 
d_query_tokens = word_tokenize(str(preprocess(query)))
print(d_query_tokens)

['java', 'coffee', 'mocha']


Counting frequencies of each word in the whole data set.


In [0]:
# processed_text
N = len (processed_text)
df = {}

for i in range(N):
    tokens = processed_text[i]
    for word in tokens:
        try:
            df[word].add(i)
        except:
            df[word] = {i}
    
for i in df:
    df[i] = len(df[i])

total_vocab_size = len(df)
print(total_vocab_size)

502


Compute tf-idf values for each word in every document as well as the query.

In [0]:
def compute_tf_idf(word_vocab, document_text,DF_freq):
    tf_idf = {} 
    N = 4
    tokens = document_text    
    counter = Counter(tokens)
    words_count = len(tokens)

    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = DF_freq[token]
        idf = np.log((N+1)/(df+1))
        tf_idf[token] = tf*idf
        
    return tf_idf


total_vocab = [x for x in df]

d1_tf_idf = compute_tf_idf(total_vocab,processed_text[0],df)
d2_tf_idf = compute_tf_idf(total_vocab,processed_text[1],df)
d3_tf_idf = compute_tf_idf(total_vocab,processed_text[2],df)
d4_tf_idf = compute_tf_idf(total_vocab,processed_text[3],df)
d5_tf_idf = compute_tf_idf(total_vocab,processed_text[4],df)
print(d1_tf_idf)
print(d2_tf_idf)
print(d3_tf_idf)
print(d4_tf_idf)
print(d5_tf_idf)


{'141': 0.009072185464100546, '56': 0.009072185464100546, '60': 0.009072185464100546, 'along': 0.009072185464100546, 'beliefs': 0.009072185464100546, 'bilingual': 0.009072185464100546, 'buddhist': 0.009072185464100546, 'capital': 0.009072185464100546, 'center': 0.005057679443227631, 'chain': 0.009072185464100546, 'city': 0.009072185464100546, 'colonial': 0.009072185464100546, 'core': 0.005057679443227631, 'cultures': 0.009072185464100546, 'diverse': 0.009072185464100546, 'dominant': 0.009072185464100546, 'dutch': 0.005057679443227631, 'earth': 0.009072185464100546, 'east': 0.009072185464100546, 'east–west': 0.009072185464100546, 'empires': 0.009072185464100546, 'eruptions': 0.009072185464100546, 'ethnicities': 0.009072185464100546, 'fifth': 0.009072185464100546, 'first': 0.009072185464100546, 'formed': 0.009072185464100546, 'forms': 0.009072185464100546, 'furthermore': 0.009072185464100546, 'hindu': 0.009072185464100546, 'history': 0.009072185464100546, 'home': 0.005057679443227631, 'i

Compute the cosine similarity between tf-idf vectors of each document and the query.

In [0]:
def cosine_similarity(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

Vectorize tf-idf

In [0]:
def vect_tf_idf(tf_idf,total_vocab_size):
    D = np.zeros((1, total_vocab_size))
    for i in tf_idf:
        try:
            ind = total_vocab.index(i)
            D[0][ind] = tf_idf[i]
        except:
            pass
    return D

In [0]:

D1 = vect_tf_idf(d1_tf_idf,total_vocab_size)
D1 = np.append (D1, vect_tf_idf(d2_tf_idf,total_vocab_size),axis = 0)
D1 = np.append (D1, vect_tf_idf(d3_tf_idf,total_vocab_size),axis = 0)
D1 = np.append (D1, vect_tf_idf(d4_tf_idf,total_vocab_size),axis = 0)
D1 = np.append (D1, vect_tf_idf(d5_tf_idf,total_vocab_size),axis = 0)
print(D1)


[[-0.02665127  0.01104671  0.03628874 ...  0.          0.
   0.        ]
 [-0.01797943  0.          0.         ...  0.          0.
   0.        ]
 [-0.01470917  0.00121936  0.         ...  0.          0.
   0.        ]
 [-0.01519552  0.00143964  0.         ...  0.          0.
   0.        ]
 [-0.02863594  0.          0.         ...  0.00649852  0.00649852
   0.        ]]


Finding tf-idf values for the query document

In [0]:
def gen_vector(tokens):
    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df_new = df[token]
        idf = math.log((N+1)/(df_new+1))
        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

D_query = gen_vector(d_query_tokens);
print(D_query)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [0]:
d_cosines = [] 
for d in D1:
    d_cosines.append(cosine_similarity(D_query, d))

print(d_cosines)


[0.0, 0.0, 0.0, 0.218230197731997, 0.0]


The fourth document D4 has the maximum similarity value